# Matching features
This notebook shows how we can match features between frames. This is a core component for many systems. 
For example when you want to reconstruct a model of the environment, or caluclate how the camera is moving or both at the same time, i.e., simultaneously build a model of the environment and estimating how the camera is moving.. 

A feature typically consists of a keypoints (location, i.e., where in the image) and a descriptor (what that part of the image looks like). When performing matching one often uses the descriptor first. In ideal cases the descriptor is so good that it can directly generate only correct matches. In most pratcical cases this is not true and we need to look for the correct matches. 

In the code below we will make use of the ORB feature. ORB is a combination of the FAST detector and the BRIEF descriptor. One of the benefits of ORB is that it is fast to compute and match.
https://www.researchgate.net/publication/221111151_ORB_an_efficient_alternative_to_SIFT_or_SURF

We make use of David Lowe's ratio test which he defined when he introduced the SIFT feature in https://www.cs.ubc.ca/~lowe/papers/ijcv04.pdf. The idea is that we want to only use matches where two points A and B in two images are much better matches than A and the second best match in the second image. 

In [ ]:
import numpy as np
import cv2 as cv

In [ ]:
# Load the image as gray scale images
img1 = cv.imread('test_images/patrics_foot_1.jpg',0)
img2 = cv.imread('test_images/patrics_foot_2.jpg',0)
print("The dimension of img1 is " + repr(img1.shape))
print("The dimension of img2 is " + repr(img2.shape))

# Initiate a feature detector
detector = cv.ORB_create()

# Find the keypoints and descriptors
kp1, des1 = detector.detectAndCompute(img1,None)
kp2, des2 = detector.detectAndCompute(img2,None)

# We use a brute force matcher with default params
bf = cv.BFMatcher()
matches = bf.knnMatch(des1,des2, k=2)

# Apply David Lowe's ratio test to remove bad matches
# https://www.cs.ubc.ca/~lowe/papers/ijcv04.pdf
good = []
for m,n in matches:
    if m.distance < 0.75 * n.distance:
        good.append([m])
        
# Draw the matches, both all of them and the good ones
image_matches_all = cv.drawMatchesKnn(img1,kp1,img2,kp2,matches,None)
image_matches_good = cv.drawMatchesKnn(img1,kp1,img2,kp2,good,None)

cv.imshow("All matches between images", image_matches_all)
cv.imshow("Good matches between images", image_matches_good)

cv.waitKey(0)
cv.destroyAllWindows()